# British Airways Lounge Demand Estimation
# Author: Hayley M
# Year: 2026
#
# Objective:
# Estimate airport lounge demand based on passenger tiers, flight timing, and route characteristics.

## British Airways - Lounge Eligibility Model Notebook 

### #1 Business Context
British Airways requires a scalable way to forecast lounge eligibility across future flight schedules at Heathrow Terminal 3.
Because future schedules may change (fleet mix, markets, timing), the model must rely on broad, reusable groupings rather than flight-level details.

This notebook:

- Loads and audits the flight schedule
- Engineers features relevant to lounge eligibility
- Tests alternative grouping strategies
- Selects the grouping with the strongest operational insight
- Supports generation of a reusable lookup table

### #2 Load Data

In [1]:
import pandas as pd
import numpy as np

# Load dataset
flights = pd.read_excel('British Airways Summer Schedule Dataset - Forage Data Science Task 1.xlsx')

### #3 Initial Data Audit
Understanding dataset shape, structure and variable types.

In [2]:
#Basic structure
flights.shape
flights.info()
flights.head(10)

<class 'pandas.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   FLIGHT_DATE           10000 non-null  datetime64[us]
 1   FLIGHT_TIME           10000 non-null  object        
 2   TIME_OF_DAY           10000 non-null  str           
 3   AIRLINE_CD            10000 non-null  str           
 4   FLIGHT_NO             10000 non-null  str           
 5   DEPARTURE_STATION_CD  10000 non-null  str           
 6   ARRIVAL_STATION_CD    10000 non-null  str           
 7   ARRIVAL_COUNTRY       10000 non-null  str           
 8   ARRIVAL_REGION        10000 non-null  str           
 9   HAUL                  10000 non-null  str           
 10  AIRCRAFT_TYPE         10000 non-null  str           
 11  FIRST_CLASS_SEATS     10000 non-null  int64         
 12  BUSINESS_CLASS_SEATS  10000 non-null  int64         
 13  ECONOMY_SEATS         10000 

,FLIGHT_DATE,FLIGHT_TIME,TIME_OF_DAY,AIRLINE_CD,FLIGHT_NO,DEPARTURE_STATION_CD,ARRIVAL_STATION_CD,ARRIVAL_COUNTRY,ARRIVAL_REGION,HAUL,AIRCRAFT_TYPE,FIRST_CLASS_SEATS,BUSINESS_CLASS_SEATS,ECONOMY_SEATS,TIER1_ELIGIBLE_PAX,TIER2_ELIGIBLE_PAX,TIER3_ELIGIBLE_PAX
0,2025-09-02,14:19:00,Afternoon,BA,BA5211,LHR,LAX,USA,North America,LONG,B777,8,49,178,0,10,38
1,2025-06-10,06:42:00,Morning,BA,BA7282,LHR,LAX,USA,North America,LONG,B777,8,49,178,0,7,28
2,2025-10-27,15:33:00,Afternoon,BA,BA1896,LHR,FRA,Germany,Europe,SHORT,A320,0,17,163,0,11,40
3,2025-06-15,18:29:00,Evening,BA,BA5497,LHR,IST,Turkey,Europe,SHORT,A320,0,8,172,0,16,54
4,2025-08-25,20:35:00,Evening,BA,BA1493,LHR,FRA,Germany,Europe,SHORT,A320,0,13,167,0,6,27
5,2025-07-12,19:08:00,Evening,BA,BA4954,LHR,VIE,Austria,Europe,SHORT,A320,0,20,160,0,6,27
6,2025-09-16,21:44:00,Evening,BA,BA9840,LHR,AMS,Netherlands,Europe,SHORT,A320,0,20,160,0,16,55
7,2025-04-24,14:50:00,Afternoon,BA,BA7116,LHR,ORD,USA,North America,LONG,A350,0,56,275,1,12,42
8,2025-05-17,12:13:00,Lunchtime,BA,BA4424,LHR,FRA,Germany,Europe,SHORT,A320,0,17,163,0,5,23
9,2025-05-01,09:55:00,Morning,BA,BA4878,LHR,IST,Turkey,Europe,SHORT,A320,0,4,176,1,11,39


#### Check Category Levels

In [3]:
#Inspect variable values
for col in ["ARRIVAL_REGION", "TIME_OF_DAY", "HAUL"]:
    vals = sorted(flights[col].dropna().unique())
    print(f"{col} ({len(vals)}): {vals}\n")

ARRIVAL_REGION (4): ['Asia', 'Europe', 'Middle East', 'North America']

TIME_OF_DAY (4): ['Afternoon', 'Evening', 'Lunchtime', 'Morning']

HAUL (2): ['LONG', 'SHORT']



#### Basic Value Distribution

In [4]:
#Frequency counts
print(flights["ARRIVAL_REGION"].value_counts())
print(flights["TIME_OF_DAY"].value_counts())
print(flights["HAUL"].value_counts())

ARRIVAL_REGION
Europe           5975
North America    2658
Middle East       688
Asia              679
Name: count, dtype: int64
TIME_OF_DAY
Morning      3530
Evening      2973
Afternoon    2305
Lunchtime    1192
Name: count, dtype: int64
HAUL
SHORT    5975
LONG     4025
Name: count, dtype: int64


#### Zero and Missing Checks
To ensure variables are clean and suitable for grouping before building the lookup model. 

In [5]:
(flights == 0).any()

FLIGHT_DATE             False
FLIGHT_TIME             False
TIME_OF_DAY             False
AIRLINE_CD              False
FLIGHT_NO               False
DEPARTURE_STATION_CD    False
ARRIVAL_STATION_CD      False
ARRIVAL_COUNTRY         False
ARRIVAL_REGION          False
HAUL                    False
AIRCRAFT_TYPE           False
FIRST_CLASS_SEATS        True
BUSINESS_CLASS_SEATS     True
ECONOMY_SEATS           False
TIER1_ELIGIBLE_PAX       True
TIER2_ELIGIBLE_PAX       True
TIER3_ELIGIBLE_PAX       True
dtype: bool

In [6]:
flights.isna().sum()

FLIGHT_DATE             0
FLIGHT_TIME             0
TIME_OF_DAY             0
AIRLINE_CD              0
FLIGHT_NO               0
DEPARTURE_STATION_CD    0
ARRIVAL_STATION_CD      0
ARRIVAL_COUNTRY         0
ARRIVAL_REGION          0
HAUL                    0
AIRCRAFT_TYPE           0
FIRST_CLASS_SEATS       0
BUSINESS_CLASS_SEATS    0
ECONOMY_SEATS           0
TIER1_ELIGIBLE_PAX      0
TIER2_ELIGIBLE_PAX      0
TIER3_ELIGIBLE_PAX      0
dtype: int64

### #4 Feature Engineering
Compute total seats because lounge eligibility percentages will be seat-weighted.

In [7]:
#Total passenger capacity per flight
flights['TOTAL_SEATS'] = (
    flights['ECONOMY_SEATS'] +
    flights['BUSINESS_CLASS_SEATS'] +
    flights['FIRST_CLASS_SEATS']
    )

### #5 Create Base Grouped Table
Starting with the most granular grouping to be explored (Haul x Time of Day x Arrival Region)

In [8]:
from itertools import combinations

group_vars = ["HAUL", "TIME_OF_DAY", "ARRIVAL_REGION"]

lounge_estimations = (
    flights
    .groupby(group_vars, as_index=False)
    .agg(
        N_FLIGHTS=("FLIGHT_NO", "size"),   
        TOTAL_TIER_1=("TIER1_ELIGIBLE_PAX", "sum"),
        TOTAL_TIER_2=("TIER2_ELIGIBLE_PAX", "sum"),
        TOTAL_TIER_3=("TIER3_ELIGIBLE_PAX", "sum"),
        TOTAL_SEATS=("TOTAL_SEATS", "sum"),
    )
)

# Combined eligible passengers - for context
lounge_estimations["TOTAL_ELIGIBLE"] = (
    lounge_estimations["TOTAL_TIER_1"] +
    lounge_estimations["TOTAL_TIER_2"] +
    lounge_estimations["TOTAL_TIER_3"]
)

lounge_estimations

,HAUL,TIME_OF_DAY,ARRIVAL_REGION,N_FLIGHTS,TOTAL_TIER_1,TOTAL_TIER_2,TOTAL_TIER_3,TOTAL_SEATS,TOTAL_ELIGIBLE
0,LONG,Afternoon,Asia,152,75,1280,4805,44373,6160
1,LONG,Afternoon,Middle East,159,100,1259,4800,47092,6159
2,LONG,Afternoon,North America,628,370,5059,19301,183271,24730
3,LONG,Evening,Asia,206,120,1602,6160,59648,7882
4,LONG,Evening,Middle East,201,130,1523,5854,58756,7507
5,LONG,Evening,North America,783,486,6160,23620,228764,30266
6,LONG,Lunchtime,Asia,75,33,553,2172,21676,2758
7,LONG,Lunchtime,Middle East,79,43,607,2329,22490,2979
8,LONG,Lunchtime,North America,281,147,2258,8593,81936,10998
9,LONG,Morning,Asia,246,151,1890,7293,71658,9334


### #6 Explore Alternative Grouping Strategies
A simple way to determine which grouping best explains lounge eligibility variation.
- A good grouping shows meaningful variance between categories
- It is also simple enough to apply to future changes to schedules, flight numbers, destinations etc. 

All compbinations of Haul, Time of Day and Arrival Region were tested. 

#### Computer percentage-point ranges per grouping
Higher range = stronger variance and more helpful for predictions.

In [9]:
all_results = []

for i in range(1, len(group_vars) + 1):
    
    for group_combo in combinations(group_vars, i):
        group_name = " × ".join(group_combo)

        grouped = (
            lounge_estimations
            .groupby(list(group_combo), as_index=False)
            .agg({
                "TOTAL_SEATS": "sum",
                "TOTAL_TIER_1": "sum",
                "TOTAL_TIER_2": "sum",
                "TOTAL_TIER_3": "sum",
                "N_FLIGHTS": "sum"
            })
        )

        grouped["PCT_TIER_1"] = 100 * grouped["TOTAL_TIER_1"] / grouped["TOTAL_SEATS"]
        grouped["PCT_TIER_2"] = 100 * grouped["TOTAL_TIER_2"] / grouped["TOTAL_SEATS"]
        grouped["PCT_TIER_3"] = 100 * grouped["TOTAL_TIER_3"] / grouped["TOTAL_SEATS"]

        all_results.append({
            "GROUPING": group_name,
            "N_GROUPS": len(grouped),
            "TIER_1_RANGE_pp": grouped["PCT_TIER_1"].max() - grouped["PCT_TIER_1"].min(),
            "TIER_2_RANGE_pp": grouped["PCT_TIER_2"].max() - grouped["PCT_TIER_2"].min(),
            "TIER_3_RANGE_pp": grouped["PCT_TIER_3"].max() - grouped["PCT_TIER_3"].min(),
        })

exploration_table = pd.DataFrame(all_results).round(2)
exploration_table

,GROUPING,N_GROUPS,TIER_1_RANGE_pp,TIER_2_RANGE_pp,TIER_3_RANGE_pp
0,HAUL,2,0.14,1.66,6.38
1,TIME_OF_DAY,4,0.02,0.17,0.58
2,ARRIVAL_REGION,4,0.15,1.71,6.56
3,HAUL × TIME_OF_DAY,8,0.22,1.87,7.01
4,HAUL × ARRIVAL_REGION,4,0.15,1.71,6.56
5,TIME_OF_DAY × ARRIVAL_REGION,16,0.24,2.00,7.31
6,HAUL × TIME_OF_DAY × ARRIVAL_REGION,16,0.24,2.00,7.31


### #7 Interpretation of Grouping Results
- Time of Day alone provides very weak segmentation.

- Arrival Region adds relatively little difference for the addition of 4 more categories. 

- Haul × Time of day has strong differentiation without excessive complexity.

Therefore, this becomes our chosen grouping.

### #8 Build Final Lookup Table
Using the chosen grouping - Haul x Time of Day

In [10]:
GROUPED_2 = ['HAUL', 'TIME_OF_DAY']

In [11]:
lounge_estimations_2 = (
    flights
    .groupby(GROUPED_2, as_index=False)
    .agg(
        N_FLIGHTS=("FLIGHT_NO", "size"),   
        TOTAL_TIER_1=("TIER1_ELIGIBLE_PAX", "sum"),
        TOTAL_TIER_2=("TIER2_ELIGIBLE_PAX", "sum"),
        TOTAL_TIER_3=("TIER3_ELIGIBLE_PAX", "sum"),
        TOTAL_SEATS=("TOTAL_SEATS", "sum"),
    )
)

for tier in ["1", "2", "3"]:
    lounge_estimations_2[f"PCT_TIER_{tier}"] = (
        lounge_estimations_2[f"TOTAL_TIER_{tier}"] /
        lounge_estimations_2["TOTAL_SEATS"] * 100
    )

lounge_estimations_2

,HAUL,TIME_OF_DAY,N_FLIGHTS,TOTAL_TIER_1,TOTAL_TIER_2,TOTAL_TIER_3,TOTAL_SEATS,PCT_TIER_1,PCT_TIER_2,PCT_TIER_3
0,LONG,Afternoon,939,545,7598,28906,274736,0.198372,2.765564,10.521373
1,LONG,Evening,1190,736,9285,35634,347168,0.212001,2.674498,10.264195
2,LONG,Lunchtime,435,223,3418,13094,126102,0.176841,2.710504,10.383658
3,LONG,Morning,1461,876,11897,45356,426563,0.205362,2.789037,10.632896
4,SHORT,Afternoon,1366,840,10606,40785,245880,0.341630,4.313486,16.587360
5,SHORT,Evening,1783,1054,14287,54546,320940,0.328410,4.451611,16.995700
6,SHORT,Lunchtime,757,537,6198,23539,136260,0.394100,4.548657,17.275062
7,SHORT,Morning,2069,1266,16230,62354,372420,0.339939,4.357983,16.742925


### #10 Summary and Business Justification
Haul x Time of Day will be used to segment flights because:
- only 8 segments - simple and easy to apply
- shows strong differences in Tier 3 demand with around 7 percentage points difference
- reflects meaningful travel behaviour

We chose not to include Arrival Region because: 
- increases complexity (16 groups)
- marginal increase percentage point difference

##### This information will be used to build a lookup table that British Airways can use to estimate lounge demand, even with incomplete schedule information. 